In [1]:
import json
import gensim
import nltk


Using Theano backend.


ImportError: No module named 'lexicon'

In [76]:
from collections import defaultdict

In [16]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/bkovarsky/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
with open("lentaruofficial.texts.json", "r") as inpf:
    data = json.load(inpf)

In [25]:
import re

In [ ]:
re.sub()

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [24]:
stemmer = nltk.stem.snowball.RussianStemmer()

In [ ]:
stemmer = s

In [36]:
s = ""

In [52]:
def word_tokenize_filt(sent, do_lowercase=False, do_stem=False):
    words = []
    for word in word_tokenize(sent):
        word = re.sub('[\"\«\»]', "", word)
        if not word:
            continue
        if do_lowercase:
            word = word.lower()
        if do_stem:
            word = stemmer.stem(word)
        words.append(word)
    return words

In [53]:
def preprocess_data(data, do_lowercase=False, do_stem=False):
    for i in range(len(data)):
        text = data[i]["text"]
        text = re.sub("<[^>]*>", "", text) # remove tags
        data[i]["processed_text"] = [word_tokenize_filt(sent, do_lowercase, do_stem)
                                         for sent in sent_tokenize(data[i]["text"])]
    return data


In [57]:
stemmer.stem("рассказывает")

'рассказыва'

In [58]:
sent = "По словам Путина, отказаться от своей"

In [59]:
s = sent.lower()

In [60]:
p = "по\sсловам\s(?P<source>[\w\.\-\«\»]+)"

In [61]:
d = re.search(p, s)

In [66]:
d.group("source")

'путина'

In [74]:
SOURCE_PATTERNS = (
    "по\sсловам\s(?P<source>[\w\.\-\«\»]+)",
    "по\sданным\s(?P<source>[\w\.\-\«\»]+)",
    "по\sсообщению\s(?P<source>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<source>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказал\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)",
    "сообщает\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "сообщает\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "сообщил\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "сообщает\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
#     "сообщил\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "сообщает\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "рассказал\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "рассказывает\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)\sв\sбеседе\sс\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)\sв\sбеседе\sс\sPubliser",
    "в\s(?P<source>[\w\.\-\«\»]+)\sрассказали\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sрассказывают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sсообщили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sотметили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sговорят\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sсообщают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sотмечают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sобяснили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sобъясняют\,\sчто",
    "как\sотмечают\sв\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотметили\sв\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотметил\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотмечает\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sрассказал\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sрассказывает\s(?P<source>[\w\.\-\«\»]+)\,",
    "подчеркнул\s(?P<source>[\w\.\-\«\»]+)",
    "подчеркивает\s(?P<source>[\w\.\-\«\»]+)",
    "объяснил\s(?P<source>[\w\.\-\«\»]+)",
    "объяснили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "пояснили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "уточнили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "(?P<source>[\w\.\-\«\»]+)\sобъяснил\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sподчеркнул\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sпояснил\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sуточнил\,\sчто",
        )


def get_sources(data):
    sources = []
    for entry in data:
        for sent in sent_tokenize(entry["text"]):
            for pattern in SOURCE_PATTERNS:
                match = re.search(pattern, sent.lower())
                if match:
                    sources.append(match.group("source"))
    return sources

In [84]:
def make_word_id_mappings(data):
    counts = defaultdict(int)
    for entry in data:
        for sent in entry["processed_text"]:
            for word in sent:
                counts[word] += 1
    counts = sorted(list(counts.items()), key=lambda x: x[1], reverse=True)
    word_to_id = dict()
    id_to_word = []
    
    for i, (word, _) in enumerate(counts):
        word_to_id[word] = i
        id_to_word.append(word)
    
    return counts, word_to_id, id_to_word
    
    

In [85]:
counts, word_to_id, id_to_word = make_word_id_mappings(data)